In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

base_dir = 'C:/Data'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)  

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  
    batch_size=32,  
    class_mode='binary',
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),  
    batch_size=32,
    class_mode='binary',
    shuffle=False,  
    seed=42
)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, epochs=3, validation_data=test_generator)

model.save('C:/Users/AliMa/Downloads/DeepFake_ImageDetection-main/DeepFake_ImageDetection-main/Models/CNN_base.h5')

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

predictions = model.predict(test_generator)
predicted_classes = np.round(predictions).flatten()  

true_labels = test_generator.classes

report = classification_report(true_labels, predicted_classes)
print(report)


Found 93853 images belonging to 2 classes.
Found 30794 images belonging to 2 classes.


c:\Users\AliMa\.conda\envs\deepFake\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/3


c:\Users\AliMa\.conda\envs\deepFake\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2933/2933 ━━━━━━━━━━━━━━━━━━━━ 1166s 396ms/step - accuracy: 0.7806 - loss: 0.4977 - val_accuracy: 0.7877 - val_loss: 0.4283
Epoch 2/3
2933/2933 ━━━━━━━━━━━━━━━━━━━━ 1120s 381ms/step - accuracy: 0.8617 - loss: 0.3126 - val_accuracy: 0.8283 - val_loss: 0.3880
Epoch 3/3
2933/2933 ━━━━━━━━━━━━━━━━━━━━ 1119s 381ms/step - accuracy: 0.9194 - loss: 0.2050 - val_accuracy: 0.8612 - val_loss: 0.4383


963/963 ━━━━━━━━━━━━━━━━━━━━ 43s 45ms/step - accuracy: 0.9125 - loss: 0.2739
Test Accuracy: 0.8612067103385925


963/963 ━━━━━━━━━━━━━━━━━━━━ 43s 45ms/step
              precision    recall  f1-score   support

           0       0.92      0.90      0.91     24765
           1       0.63      0.70      0.66      6029

    accuracy                           0.86     30794
   macro avg       0.78      0.80      0.79     30794
weighted avg       0.87      0.86      0.86     30794



In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    img = cv2.resize(img, (64, 64))  
    img = img / 255.0  
    return np.expand_dims(img, axis=0)  # 

model = load_model('C:/Users/AliMa/Downloads/DeepFake_ImageDetection-main/DeepFake_ImageDetection-main/Models/CNN_base.h5')  

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

image_path = "C:/test/fake3.jpg" 
preprocessed_image = preprocess_image(image_path)

predictions = model.predict(preprocessed_image)

class_names = ['Real', 'Fake'] 
predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)
print("Confidence:", predictions[0][np.argmax(predictions)])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted class: Real
Confidence: 3.7310233e-12
